In [2]:
# Install necessary library
!pip install  transformers torch ipywidgets newspaper3k lxml_html_clean


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.2 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=d6f339a22b2a09ff84923cb1b51319b4d314beeec2d79125fa3e86c51edf7945
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=e1ca6441b0ef9a8f7502411136a3210da2f3213b7c490f06769b710d775b063b
  Stored in di

In [3]:
#Import necessary library
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    BartForConditionalGeneration, BartTokenizer,
    PegasusForConditionalGeneration, PegasusTokenizer
)
import torch
from IPython.display import display
import ipywidgets as widgets
from newspaper import Article


In [4]:
#Set CUDA GPU if availavle if not then defult CPU and print ruuning one.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [5]:
#Store two models and their tokenizers, you may more models here
models_tok = {
        'Pegsus Large': {
        'model': PegasusForConditionalGeneration.from_pretrained('google/pegasus-large').to(device),
        'tok': PegasusTokenizer.from_pretrained('google/pegasus-large')},

     'Bart Large': {
        'model': BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device),
        'tok': BartTokenizer.from_pretrained('facebook/bart-large-cnn') }
        }


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [10]:
# define for generating summary using specified model

def s_text(text, m_name, max_length=150, min_length=40):
    if not text:
        return "Please enter text to summarize."

    m_info = models_tok[m_name]
    model = m_info['model']
    tokenizer = m_info['tok']
    inputs = tokenizer.encode(
        text, return_tensors="pt", max_length=5000, truncation=True
    ).to(device)

    summary_ids = model.generate(
        inputs,
        max_length=max_length,
        min_length=min_length,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [15]:
# Widgets GPU
url_input = widgets.Text(description='Article URL:', placeholder='Paste URL here...', layout=widgets.Layout(width='75%'))
t_input = widgets.Textarea(description='News Article Text:', placeholder='Or paste the article text here...', layout=widgets.Layout(width='75%', height='200px'))
s_output = widgets.Textarea(description='Summary:', layout=widgets.Layout(width='75%', height='200px'), disabled=True)
m_selector = widgets.Dropdown(
    options=list(models_tok.keys()),
    value='Pegsus Large',
    description='Select Model:',
    disabled=False,
)

sum_b = widgets.Button(description='Summarize', style=widgets.ButtonStyle(button_color='lightblue'))
clr_b = widgets.Button(description='Clear', style=widgets.ButtonStyle(button_color='lightgray'))
load_ind = widgets.Label(value='', layout=widgets.Layout(margin='0 0 0 20px'))



# Button Functions
def on_sum_button_click(b):
    load_ind.value = 'Processing...'
    s_output.value = ''
    text_to_summarize = ""

    if url_input.value:
        try:
            article = Article(url_input.value)
            article.download()
            article.parse()
            text_to_summarize = article.text
        except Exception as e:
            s_output.value = f"Error scraping article: {e}"
            load_ind.value = ''
            return
    else:
        text_to_summarize = t_input.value

    if not text_to_summarize.strip():
        s_output.value = "No content found. Try pasting text manually."
        load_ind.value = ''
        return

    summary = s_text(text_to_summarize, m_selector.value)
    s_output.value = summary
    load_ind.value = ''

def on_clear_button_click(b):
    url_input.value = ''
    t_input.value = ''
    s_output.value = ''

sum_b.on_click(on_sum_button_click)
clr_b.on_click(on_clear_button_click)

# Display UI
controls = widgets.VBox([m_selector, url_input, t_input])
buttons = widgets.HBox([sum_b, clr_b, load_ind])
display(controls, buttons, s_output)


Textarea(value='', description='Summary:', disabled=True, layout=Layout(height='200px', width='75%'))